In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

In [2]:
! ls

data
images
mid_terms.ipynb
mid_terms_sm.ipynb
requirements.txt
workshop_1_linear_regression.ipynb
workshop_2_logistic_regression_lda_qda_nb.ipynb
workshop_3_resampling_methods.ipynb
workshop_4_model_selection_and_regularizations.ipynb
workshop_5_beyond_linearity.ipynb


In [3]:
! pip install -U -r requirements.txt

## Update repository

In [4]:
! git pull

Already up to date.


## Add import path

In [5]:
import os
import sys
import gc

In [6]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
del module_path

## Organize imports

In [8]:
import multiprocessing
from pathlib import Path

In [9]:
import seaborn as sns

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, PolynomialFeatures, 
                                   SplineTransformer)
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis, 
                                           QuadraticDiscriminantAnalysis)
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import (mean_absolute_error, mean_squared_error, 
                             classification_report, confusion_matrix)

#### Number of CPU cores

In [11]:
workers = multiprocessing.cpu_count()
workers

8

## Initialize path

In [12]:
DATA = Path('data')
PATH = DATA / 'log_regr_lda_qda_np'
LR_PATH = DATA / 'linear_regression'
PUMPKIN_DIR = PATH / 'Pumpkin_Seeds_Dataset'
IRIS_DIR = PATH / 'iris'
PUMPKIN_DIR.mkdir(exist_ok=True, parents=True)
IRIS_DIR.mkdir(exist_ok=True, parents=True)

In [13]:
! ls

data
images
mid_terms.ipynb
mid_terms_sm.ipynb
requirements.txt
workshop_1_linear_regression.ipynb
workshop_2_logistic_regression_lda_qda_nb.ipynb
workshop_3_resampling_methods.ipynb
workshop_4_model_selection_and_regularizations.ipynb
workshop_5_beyond_linearity.ipynb


## Load IRIS dataset

In [14]:
SEED = 2022

In [15]:
iris_url = 'https://www.kaggle.com/datasets/uciml/iris/download?datasetVersionNumber=2'

#### Load dataset

In [16]:
! ls {IRIS_DIR}

Iris.csv        database.sqlite


In [17]:
df = pd.read_csv(IRIS_DIR / 'Iris.csv')

In [18]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [19]:
df['Species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

In [20]:
y = df['Species']
X = df.drop(columns=['Id', 'Species'], axis=1)
X.shape, y.shape, df['Species'].value_counts()

((150, 4),
 (150,),
 Iris-setosa        50
 Iris-versicolor    50
 Iris-virginica     50
 Name: Species, dtype: int64)

```python
# #define predictor and response variables
X = df[['s_length', 's_width', 'p_length', 'p_width']]
y = df['species']
X.shape, y.shape
```

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED)

## Generate polynomial features

In [22]:
poly = PolynomialFeatures(degree=2)

In [23]:
X_train_pl = poly.fit_transform(X_train)
X_test_pl = poly.transform(X_test)

In [24]:
feature_names_pl = poly.get_feature_names_out(input_features=X_train.columns)
feature_names_pl

array(['1', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm',
       'PetalWidthCm', 'SepalLengthCm^2', 'SepalLengthCm SepalWidthCm',
       'SepalLengthCm PetalLengthCm', 'SepalLengthCm PetalWidthCm',
       'SepalWidthCm^2', 'SepalWidthCm PetalLengthCm',
       'SepalWidthCm PetalWidthCm', 'PetalLengthCm^2',
       'PetalLengthCm PetalWidthCm', 'PetalWidthCm^2'], dtype=object)

In [26]:
X_train.shape, X_test.shape, X_train_pl.shape, X_test_pl.shape

((120, 4), (30, 4), (120, 15), (30, 15))

## Initialize the model

In [27]:
lda = LinearDiscriminantAnalysis()

## Train model with polynomial features

In [28]:
lda_pl = lda.fit(X_train_pl, y_train)

In [29]:
y_pred_pl = lda_pl.predict(X_test_pl)

In [30]:
cr = classification_report(y_test, y_pred_pl)
print(cr)

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         9
Iris-versicolor       0.89      1.00      0.94         8
 Iris-virginica       1.00      0.92      0.96        13

       accuracy                           0.97        30
      macro avg       0.96      0.97      0.97        30
   weighted avg       0.97      0.97      0.97        30



#### Forward selection

In [31]:
X_train_pl.shape

(120, 15)

In [ ]:
sfs_forward = SFS(lda, 
                  n_features_to_select=2, 
                  direction='forward',
                  cv=10,
                  n_jobs=workers)

sfs_forward = sfs_forward.fit(X_train_pl, y_train)
sfs_forward

In [ ]:
X_train_fw = sfs_forward.transform(X_train)
X_test_fw = sfs_forward.transform(X_test)
X_train_fw.shape

In [48]:
X_train.columns

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object')

In [49]:
sfs_forward.get_feature_names_out(X_train.columns)

array(['SepalWidthCm', 'PetalWidthCm'], dtype=object)

In [33]:
lda_fw = LinearDiscriminantAnalysis()

In [34]:
lda_fw = lda_fw.fit(X_train_fw, y_train)

In [35]:
y_pred_fw = lda_fw.predict(X_test_fw)

In [36]:
cr = classification_report(y_test, y_pred_fw)
print(cr)

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         9
Iris-versicolor       0.80      1.00      0.89         8
 Iris-virginica       1.00      0.85      0.92        13

       accuracy                           0.93        30
      macro avg       0.93      0.95      0.94        30
   weighted avg       0.95      0.93      0.93        30



#### Backward selection\

In [33]:
lda = LinearDiscriminantAnalysis()

In [34]:
X_train_pl.shape

(120, 15)

In [35]:
sfs_backward = SFS(lda, 
                  n_features_to_select=2, 
                  direction='backward',
                  cv=10,
                  n_jobs=workers)

sfs_backward = sfs_backward.fit(X_train_pl, y_train)
sfs_backward

SequentialFeatureSelector(cv=10, direction='backward',
                          estimator=LinearDiscriminantAnalysis(),
                          n_features_to_select=2, n_jobs=8)

In [36]:
X_train_bw = sfs_backward.transform(X_train_pl)
X_test_bw = sfs_backward.transform(X_test_pl)
X_train_bw.shape

(120, 2)

In [37]:
sfs_backward.get_feature_names_out(input_features=feature_names_pl)

array(['PetalWidthCm', 'SepalWidthCm PetalWidthCm'], dtype=object)

In [38]:
lda_bw = LinearDiscriminantAnalysis()

In [39]:
lda_bw = lda_bw.fit(X_train_bw, y_train)

In [40]:
y_pred_bw = lda_bw.predict(X_test_bw)

In [41]:
cr = classification_report(y_test, y_pred_bw)
print(cr)

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         9
Iris-versicolor       0.70      0.88      0.78         8
 Iris-virginica       0.91      0.77      0.83        13

       accuracy                           0.87        30
      macro avg       0.87      0.88      0.87        30
   weighted avg       0.88      0.87      0.87        30



## Ridge and Lasso regression

In [42]:
pertol_consumption_path = LR_PATH / 'petrol_consumption.csv'

In [43]:
df = pd.read_csv(pertol_consumption_path)
df

,Petrol_tax,Average_income,Paved_Highways,Population_Driver_licence(%),Petrol_Consumption
0,9.00,3571,1976,0.525,541
1,9.00,4092,1250,0.572,524
2,9.00,3865,1586,0.580,561
3,7.50,4870,2351,0.529,414
4,8.00,4399,431,0.544,410
5,10.00,5342,1333,0.571,457
6,8.00,5319,11868,0.451,344
7,8.00,5126,2138,0.553,467
8,8.00,4447,8577,0.529,464
9,7.00,4512,8507,0.552,498


In [44]:
df.shape

(48, 5)

In [45]:
print(df.describe().round(2).T)

                              count     mean      std      min      25%  \
Petrol_tax                     48.0     7.67     0.95     5.00     7.00   
Average_income                 48.0  4241.83   573.62  3063.00  3739.00   
Paved_Highways                 48.0  5565.42  3491.51   431.00  3110.25   
Population_Driver_licence(%)   48.0     0.57     0.06     0.45     0.53   
Petrol_Consumption             48.0   576.77   111.89   344.00   509.50   

                                  50%      75%       max  
Petrol_tax                       7.50     8.12     10.00  
Average_income                4298.00  4578.75   5342.00  
Paved_Highways                4735.50  7156.00  17782.00  
Population_Driver_licence(%)     0.56     0.60      0.72  
Petrol_Consumption             568.50   632.75    968.00  


#### Prepare dataset

In [46]:
y = df['Petrol_Consumption']
X = df[['Average_income', 'Paved_Highways',
       'Population_Driver_licence(%)', 'Petrol_tax']]

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=SEED)

## Generate Polynomial features

In [48]:
poly = PolynomialFeatures(degree=4)

In [49]:
X_train_pl = poly.fit_transform(X_train)
X_test_pl = poly.transform(X_test)

In [50]:
feature_names_pl = poly.get_feature_names_out(input_features=X_train.columns)
feature_names_pl

array(['1', 'Average_income', 'Paved_Highways',
       'Population_Driver_licence(%)', 'Petrol_tax', 'Average_income^2',
       'Average_income Paved_Highways',
       'Average_income Population_Driver_licence(%)',
       'Average_income Petrol_tax', 'Paved_Highways^2',
       'Paved_Highways Population_Driver_licence(%)',
       'Paved_Highways Petrol_tax', 'Population_Driver_licence(%)^2',
       'Population_Driver_licence(%) Petrol_tax', 'Petrol_tax^2',
       'Average_income^3', 'Average_income^2 Paved_Highways',
       'Average_income^2 Population_Driver_licence(%)',
       'Average_income^2 Petrol_tax', 'Average_income Paved_Highways^2',
       'Average_income Paved_Highways Population_Driver_licence(%)',
       'Average_income Paved_Highways Petrol_tax',
       'Average_income Population_Driver_licence(%)^2',
       'Average_income Population_Driver_licence(%) Petrol_tax',
       'Average_income Petrol_tax^2', 'Paved_Highways^3',
       'Paved_Highways^2 Population_Driver_licenc

In [51]:
X_train.shape, X_train_pl.shape

((38, 4), (38, 70))

In [52]:
std_scaller = StandardScaler()
X_train_pl = std_scaller.fit_transform(X_train_pl)
X_test_pl = std_scaller.transform(X_test_pl)

#### Ridge

In [53]:
ridge_rg = Ridge(alpha=100.08)
ridge_rg.fit(X_train_pl, y_train)

Ridge(alpha=100.08)

In [54]:
ridge_rg.intercept_

586.8157894736842

In [55]:
ridge_rg.coef_

array([ 0.        , -2.10250677,  0.28957979,  5.35046514, -3.48465873,
       -1.97910479, -0.28735971,  1.60920851, -4.01930384, -0.51699779,
        0.90701634, -0.12913344,  5.7223179 , -0.28180499, -3.17983511,
       -1.79760899, -0.80321086,  0.49632535, -3.42995055, -0.86066365,
        0.38360057, -0.75224681,  3.73326282, -1.26455031, -3.76987467,
       -0.23295432, -0.29617984, -0.97896859,  1.64363065,  0.53693416,
       -0.49961944,  6.08804461,  2.40806828, -1.85954463, -2.84266327,
       -1.57045653, -1.21134692,  0.15158578, -2.93857089, -1.19891391,
       -0.11045046, -1.2547003 ,  2.55523381, -1.40583406, -3.58838867,
       -0.48969576, -0.62836311, -1.31645802,  1.19860937, -0.05513148,
       -1.11537306,  4.98455866,  1.30441416, -2.357977  , -3.27855312,
        0.15240799, -0.1401816 , -0.67620337, -0.01832285, -0.72682675,
       -1.37259105,  2.45942718,  1.34238985,  0.14410685, -0.74239435,
        6.44357751,  3.8755325 , -0.08222846, -2.14929811, -2.49

In [56]:
model_coefficients = ridge_rg.coef_

coefficients_df = pd.DataFrame(data = model_coefficients, 
                              index = feature_names_pl, 
                              columns = ['Coefficient value'])
print(coefficients_df)

                                             Coefficient value
1                                                     0.000000
Average_income                                       -2.102507
Paved_Highways                                        0.289580
Population_Driver_licence(%)                          5.350465
Petrol_tax                                           -3.484659
...                                                        ...
Population_Driver_licence(%)^4                        6.443578
Population_Driver_licence(%)^3 Petrol_tax             3.875533
Population_Driver_licence(%)^2 Petrol_tax^2          -0.082228
Population_Driver_licence(%) Petrol_tax^3            -2.149298
Petrol_tax^4                                         -2.493301

[70 rows x 1 columns]


#### Inference with the model

In [57]:
y_pred_pl = ridge_rg.predict(X_test_pl)
y_pred_pl

array([474.4851455 , 680.98317028, 539.37429978, 542.03010383,
       638.81542413, 422.83703026, 530.44920839, 575.14415063,
       578.59319234, 546.15030112])

In [58]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_pl})
print(results)

    Actual   Predicted
22     464  474.485146
39     968  680.983170
25     566  539.374300
3      414  542.030104
40     587  638.815424
6      344  422.837030
7      467  530.449208
9      498  575.144151
43     591  578.593192
34     487  546.150301


#### Evaluate the model

In [59]:
mae = mean_absolute_error(y_test, y_pred_pl)
mse = mean_squared_error(y_test, y_pred_pl)
rmse = np.sqrt(mse)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')

Mean absolute error: 79.50
Mean squared error: 12211.91
Root mean squared error: 110.51


In [60]:
actual_minus_predicted = sum((y_test - y_pred_pl)**2)
actual_minus_actual_mean = sum((y_test - y_test.mean())**2)
r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
print('R²:', r2)

R²: 0.5278057772858207


In [61]:
ridge_rg.score(X_test_pl, y_test)

0.5278057772858207

In [62]:
ridge_rg.score(X_train_pl, y_train)

0.623788782003802

#### Lasso

In [63]:
lasso_rg = Lasso(alpha=1.8)
lasso_rg.fit(X_train_pl, y_train)

Lasso(alpha=1.8)

In [64]:
lasso_rg.intercept_

586.8157894736842

In [65]:
lasso_rg.coef_

array([  0.        , -12.81279436,  -0.        ,   0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
       -21.01371796,  -0.        ,  -0.        ,   0.        ,
         0.        , -24.14642479,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,   0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,   0.        ,   0.        ,
        -0.        ,  -0.        ,   0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -3.13934643,
        -0.        ,  -0.        ,   0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,   0.        ,   0.        ,
        -0.        ,  -0.        ,   0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -6.11136048,
        -0.        ,  -0.        ,  -0.        ,   0.  

In [66]:
model_coefficients = lasso_rg.coef_

coefficients_df = pd.DataFrame(data = model_coefficients, 
                              index = feature_names_pl, 
                              columns = ['Coefficient value'])
print(coefficients_df)

                                             Coefficient value
1                                                     0.000000
Average_income                                      -12.812794
Paved_Highways                                       -0.000000
Population_Driver_licence(%)                          0.000000
Petrol_tax                                           -0.000000
...                                                        ...
Population_Driver_licence(%)^4                       71.587989
Population_Driver_licence(%)^3 Petrol_tax            -0.000000
Population_Driver_licence(%)^2 Petrol_tax^2          -0.000000
Population_Driver_licence(%) Petrol_tax^3            -0.000000
Petrol_tax^4                                          0.000000

[70 rows x 1 columns]


#### Inference with the model

In [67]:
y_pred = lasso_rg.predict(X_test_pl)
y_pred

array([450.86329313, 719.13725271, 530.69407242, 519.35676897,
       646.71282854, 418.45917599, 503.43730106, 560.22651198,
       579.65269472, 546.84686647])

In [68]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_pl})
print(results)

    Actual   Predicted
22     464  474.485146
39     968  680.983170
25     566  539.374300
3      414  542.030104
40     587  638.815424
6      344  422.837030
7      467  530.449208
9      498  575.144151
43     591  578.593192
34     487  546.150301


#### Evaluate the model

In [69]:
mae = mean_absolute_error(y_test, y_pred_pl)
mse = mean_squared_error(y_test, y_pred_pl)
rmse = np.sqrt(mse)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')

Mean absolute error: 79.50
Mean squared error: 12211.91
Root mean squared error: 110.51


In [70]:
actual_minus_predicted = sum((y_test - y_pred_pl)**2)
actual_minus_actual_mean = sum((y_test - y_test.mean())**2)
r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
print('R²:', r2)

R²: 0.5278057772858207


In [71]:
lasso_rg.score(X_test_pl, y_test)

0.6424419241829249

In [72]:
lasso_rg.score(X_train_pl, y_train)

0.7104529011707852

## Linear / polynomial splines

In [73]:
spline = SplineTransformer(n_knots=32, degree=1)

In [74]:
X_train_sp = spline.fit_transform(X_train)
X_test_sp = spline.transform(X_test)

In [75]:
feature_names_sp = spline.get_feature_names_out(input_features=X_train.columns)
feature_names_sp

array(['Average_income_sp_0', 'Average_income_sp_1',
       'Average_income_sp_2', 'Average_income_sp_3',
       'Average_income_sp_4', 'Average_income_sp_5',
       'Average_income_sp_6', 'Average_income_sp_7',
       'Average_income_sp_8', 'Average_income_sp_9',
       'Average_income_sp_10', 'Average_income_sp_11',
       'Average_income_sp_12', 'Average_income_sp_13',
       'Average_income_sp_14', 'Average_income_sp_15',
       'Average_income_sp_16', 'Average_income_sp_17',
       'Average_income_sp_18', 'Average_income_sp_19',
       'Average_income_sp_20', 'Average_income_sp_21',
       'Average_income_sp_22', 'Average_income_sp_23',
       'Average_income_sp_24', 'Average_income_sp_25',
       'Average_income_sp_26', 'Average_income_sp_27',
       'Average_income_sp_28', 'Average_income_sp_29',
       'Average_income_sp_30', 'Average_income_sp_31',
       'Paved_Highways_sp_0', 'Paved_Highways_sp_1',
       'Paved_Highways_sp_2', 'Paved_Highways_sp_3',
       'Paved_Highways_s

In [78]:
X_train.shape, X_train_sp.shape

((38, 4), (38, 128))

In [79]:
std_scaller = StandardScaler()
X_train_sp = std_scaller.fit_transform(X_train_sp)
X_test_sp = std_scaller.transform(X_test_sp)

#### Ridge

In [89]:
ridge_rg = Ridge(alpha=36.08)
ridge_rg.fit(X_train_sp, y_train)

Ridge(alpha=36.08)

In [90]:
ridge_rg.intercept_

586.8157894736842

In [91]:
ridge_rg.coef_

array([ 1.64016377,  0.        ,  0.        , -2.12693446,  2.61579471,
        1.05530638,  1.00493399, -1.87256978,  4.3451856 ,  6.85374873,
        2.07844391,  3.33458287,  5.79866499,  2.1005527 , -0.32371891,
       -2.90221771, -2.49305369,  0.62481799, -4.86712044, -8.2352274 ,
       -5.28767436, -0.92922589,  9.91990283,  8.21965093, -4.8755613 ,
        0.        , -0.23924133, -3.9452208 , -4.35738529,  4.88349966,
        5.26049189, -5.43875039, -7.52073011, -2.35985027, -4.394041  ,
        2.84083615, -3.98657816,  2.27063506,  3.73923168,  3.6265515 ,
        1.96535766,  3.81025813,  8.07977352, -3.53472883, -4.427518  ,
        2.68002775, -1.49456431, -0.17643459, -0.60085051, -4.06786081,
        1.94690898,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -4.35738529, -4.35738529,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  2.10554684, -3.54911259,
        0.        , -2.1389964 , -2.37072531, -4.40210558, -2.14

In [92]:
model_coefficients = ridge_rg.coef_

coefficients_df = pd.DataFrame(data = model_coefficients, 
                              index = feature_names_sp, 
                              columns = ['Coefficient value'])
print(coefficients_df)

                     Coefficient value
Average_income_sp_0           1.640164
Average_income_sp_1           0.000000
Average_income_sp_2           0.000000
Average_income_sp_3          -2.126934
Average_income_sp_4           2.615795
...                                ...
Petrol_tax_sp_27              0.000000
Petrol_tax_sp_28              0.000000
Petrol_tax_sp_29              0.000000
Petrol_tax_sp_30              0.000000
Petrol_tax_sp_31             -5.438750

[128 rows x 1 columns]


#### Inference with the model

In [93]:
y_pred_sp = ridge_rg.predict(X_test_sp)
y_pred_sp

array([520.98464968, 650.27087078, 601.58319712, 567.91585837,
       580.40465301, 554.46923419, 529.58223901, 514.68577948,
       579.56664944, 570.00155061])

In [94]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_sp})
print(results)

    Actual   Predicted
22     464  520.984650
39     968  650.270871
25     566  601.583197
3      414  567.915858
40     587  580.404653
6      344  554.469234
7      467  529.582239
9      498  514.685779
43     591  579.566649
34     487  570.001551


#### Evaluate the model

In [95]:
mae = mean_absolute_error(y_test, y_pred_sp)
mse = mean_squared_error(y_test, y_pred_sp)
rmse = np.sqrt(mse)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')

Mean absolute error: 95.50
Mean squared error: 18471.10
Root mean squared error: 135.91


In [96]:
R_2_test = ridge_rg.score(X_test_sp, y_test)
print(f'R² (test): {R_2_test}')

R² (test): 0.2857832051952722


In [97]:
R_2_train = ridge_rg.score(X_train_sp, y_train)
print(f'R² (train): {R_2_train}')

R² (train): 0.917918645499155
